In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import pickle

In [2]:
data_path = 'F:/projevt/Training/'
classes = {'Insect_Coloradopotatobeetle': 0, 'Insect_Fleabeetle': 1}

In [3]:
# Read the data file names and corresponding labels
data = []
labels = []
for class_name, label in classes.items():
    class_path = os.path.join(data_path, class_name)
    file_names = os.listdir(class_path)
    for file_name in file_names:
        file_path = os.path.join(class_path, file_name)
        data.append(file_path)
        labels.append(label)

In [4]:
 # Split the data into training, validation, and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)


In [5]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)

    # Background removal using thresholding
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresholded = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    image[thresholded == 0] = [0, 0, 0]  # Set background pixels to black

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Contrast improvement and histogram equalization
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    image = clahe.apply(gray)

    # Gamma correction
    gamma = 1.5
    image = np.power(image / 255.0, gamma)
    image = np.uint8(image * 255)

    # Smoothing
    image = cv2.GaussianBlur(image, (5, 5), 0)

    image = cv2.resize(image, (128, 128))
    return image



train_images = [preprocess_image(image_path) for image_path in train_data]
val_images = [preprocess_image(image_path) for image_path in val_data]
test_images = [preprocess_image(image_path) for image_path in test_data]

In [7]:
# Print the size of each set
print('Training set size:', len(train_data))
print('Validation set size:', len(val_data))
print('Testing set size:', len(test_data))

Training set size: 256
Validation set size: 64
Testing set size: 80


In [8]:
# Convert the images and labels to numpy arrays
train_images = np.array(train_images)
val_images = np.array(val_images)
test_images = np.array(test_images)
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)
test_labels = np.array(test_labels)

In [9]:
# FEATURE EXTRACTOR function
def feature_extractor(dataset):
    x_train = dataset
    image_dataset = pd.DataFrame()
    for image in range(x_train.shape[0]):
        df = pd.DataFrame()
        img = x_train[image]

        # FEATURE 1 - Pixel values
        pixel_values = img.reshape(-1)
        df['Pixel_Value'] = pixel_values

        # FEATURE 2 - Gabor filter responses
        num = 1
        kernels = []
        for theta in range(2):
            theta = theta / 4. * np.pi
            for sigma in (1, 3):
                lamda = np.pi / 4
                gamma = 0.5
                gabor_label = 'Gabor' + str(num)
                kernel = cv2.getGaborKernel((9, 9), sigma, theta, lamda, gamma, 0, ktype=cv2.CV_32F)
                kernels.append(kernel)
                fimg = cv2.filter2D(img, cv2.CV_8UC3, kernel)
                filtered_img = fimg.reshape(-1)
                df[gabor_label] = filtered_img
                num += 1

        # Append features from the current image to the dataset
        image_dataset = pd.concat([df, image_dataset])

    return image_dataset

# Extract features from training images
image_features = feature_extractor(train_images)

In [10]:
# Reshape for SVM training
n_samples = train_images.shape[0]
image_features = np.array(image_features)
image_features = np.reshape(image_features, (n_samples, -1))

In [11]:
# Define the classifier
SVM_model = SVC(kernel='rbf', random_state=42)

In [ ]:
# Fit the model on training data
SVM_model.fit(image_features, train_labels)

# Save the trained model
with open('F:/projevt/', 'wb') as file:
    pickle.dump(SVM_model, file)

# Extract features from test data and reshape
test_features = feature_extractor(test_images)
test_features = np.array(test_features)
test_features = np.reshape(test_features, (test_images.shape[0], -1))

# Predict on test data
test_predictions = SVM_model.predict(test_features)

# Convert labels back to original class names
le = LabelEncoder()
le.fit(labels)
test_predictions = le.inverse_transform(test_predictions)

